In [2]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col

In [5]:
startups = (pd.read_csv('Startup Data/startup.csv'))
startups 

,2015 Founding,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Year Ended,Surviving Establishments,Total Employment of Survivors,Survival Rates Since Birth (%),Labor Force Participation Rate (%)
1,Mar-15,"677,876","3,011,469",100,4.4
2,Mar-16,"539,701","3,073,279",79.6,5.7
3,Mar-17,"468,293","3,081,916",69.1,6.6
4,Mar-18,"416,438","3,075,769",61.4,7.4
...,...,...,...,...,...
64,Mar-23,"818,549","3,717,341",76.8,4.5
65,NaN,NaN,NaN,NaN,NaN
66,2023,NaN,NaN,NaN,NaN
67,Year Ended,Surviving Establishments,Total Employment of Survivors,Survival Rates Since Birth (%),Labor Force Participation Rate (%)


In [6]:
selected_columns = ['Year Ended' 'Surviving Establishments']
startups_numbers  = startups[selected_columns]
startups_numbers.to_csv('startup_numbers.csv', index=False)

KeyError: "None of [Index(['Year EndedSurviving Establishments'], dtype='object')] are in the [columns]"

In [ ]:
wfh = (pd.read_csv('WFH_DATA'))
wfh

In [ ]:
wfh_startups = lambda how: pd.merge(left=wfh,
                                   right=startups,
                                   on=['fyear'],
                                   how=how,indicator=True,validate="one_to_one")

In [ ]:
# Run OLS
m1 = sm_ols('wfh% ~ startups#', data=wfh_startups).fit()
m2 = sm_ols('wfh% ~ log_startups#', data=wfh_startups).fit()
m3 = sm_ols('log_wfh% ~ startups#', data=wfh_startups).fit()

# Print out multiple regression results at once
table = summary_col(results=[m1,m2,m3],
                    float_format='%0.2f',
                    stars = True,
                    model_names=['m1','m2','m3'],
                    info_dict=info_dict)

print(table)


In [ ]:
lineplot1 = sns.lineplot(data = wfh_startups,
             x='wfh%',y='startups',hue='high_leverage') #style='gsector'

In [ ]:
lineplot2 = sns.lineplot(data = wfh_startups,
             x='wfh%',y='log_startups',hue='high_leverage') #style='gsector'

In [ ]:
lineplot3 = sns.lineplot(data = wfh_startups,
             x='log_wfh%',y='startups',hue='high_leverage') #style='gsector'